In [1]:
import tweepy
import json
from itertools import islice
import os
import time
import pprint
import requests
import random
import networkx as nx
from networkx.algorithms.approximation import clique
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
from pyvis.network import Network
import itertools 

print("Librerie caricate!")


Librerie caricate!


In [44]:

pp=pprint.PrettyPrinter()

def serialize_json(folder, filename, data):
    if not os.path.exists(folder):
        os.makedirs(folder, exist_ok=True)
    with open(f"{folder}/{filename}", 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
        f.close()
    print(f"Data serialized to path: {folder}/{filename}")

def read_json(path):
    if os.path.exists(path):
        with open(path, "r", encoding="utf8") as file:
            data = json.load(file)
        print(f"Data read from path: {path}")
        return data
    else:
        print(f"No data found at path: {path}")
        return {}

In [18]:
api_key="cvoM8D7hXXxlvBXTM8aH9X2ec"
api_secret="Uk2UvH0FJY7KaDzkXYLfgiYkA1OuCwbLlGFPyodB5wcQ5bsItN"

access_token="3303466053-OLuExo5KcP8UQCVwZwmyakZs8b91Fpl2lMOUDAe"
access_secret="1lMXufu42KN8JvJjYT7c0zI3Q57CkN09BxkNXZuNQ0Dej"

bearer_token="AAAAAAAAAAAAAAAAAAAAAHsVJQEAAAAAQ4vYb83r6ueD8QvjJ4Zpx9R7Kbw%3DQuLzsmDYOvpff7lRHGXhNJSOXTFuPyOwLHZv7HPSj9WF34h1E8"

In [19]:
auth=tweepy.OAuthHandler(api_key,api_secret)
auth.set_access_token(access_token,access_secret)
api=tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)
if(api.verify_credentials):
    print("Auth success")


Auth success


In [3]:
users=["mizzaro","damiano10","miccighel_","eglu81","KevinRoitero"]
users_id=[18932422, 132646210, 15750573, 19659370, 3036907250]

In [3]:
#1. Scaricate
#codice da non eseguire a meno che non vogliate bruciarvi tutte le richieste di twitter API
followers_ids,following_ids={},{}

for utente in users_id:
    followers_utente,following_utente=[],[]
    for item in tweepy.Cursor(
        api.followers,
        id=utente,
        skip_status=True,
        include_user_entities=False
    ).items():
        json_data=item._json
        user={"id":json_data["id"]}
        time.sleep(181)
        #followers_utente.append({id:item._data["id"]})
        followers_utente.append(user)

    followers_ids[utente]=followers_utente

    for item in tweepy.Cursor(
        api.friends,
        id=utente,
        skip_status=True,
        include_user_entities=False
    ).items():
        json_data=item._json
        user={"id":json_data["id"]}
        time.sleep(181)
        following_utente.append(user)

    following_ids[utente]=following_utente

#followers_ids={"mizzaro":[123,456,789,...],"damiano10":[123,789,367,...],...}
#export to json
serialize_json("data","followers_principali.json",followers_ids)
serialize_json("data","following_principali.json",following_ids)

NameError: name 'users_id' is not defined

In [7]:
followers_ids=read_json("data_ids/finale/followers.json")
following_ids=read_json("data_ids/finale/following.json")
#Temporaneamente ho unito i followers/ing di ogni utente in due file separati, non so se è giusto

Data read from path: json_5_utenti/finale/followers.json
Data read from path: json_5_utenti/finale/following.json


In [8]:
followers_ids, following_ids = [], []

# punto 2&3
followers_ids = read_json("/followers_5_utenti.json")
following_ids = read_json("/following_5_utenti.json")

# 2
random_followers_utenti_ids, followers_of_followers_ids = {}, {}

for utente in followers_ids:
    random_followers_utenti_ids[utente] = random.sample(followers_ids[utente], 5)
# result: random_followers_utenti_ids= {"mizzaro":[{}{}{}{}{}],"damiano10":[{}...}],...}

for utente in following_ids:
    random_following_utenti_ids[utente] = random.sample(following_ids[utente], 5)
# result: random_following_utenti_ids= {"mizzaro":[{}{}{}{}{}],"damiano10":[{}...}],...}

#Download dei followers dei 10 followers degli utenti random 
for utente in random_followers_utenti_ids:
    for f in random_followers_utenti_ids[utente]:
        fof = []
        print("scarico per " + str(utente))
        for item in tweepy.Cursor(
                api.followers,
                id=f['id'],
                skip_status=True,
                include_user_entities=False
        ).items(10):
                time.sleep(10)
                json_data = item._json
                user = {"id": json_data["id"]}
                fof.append(user)
                print("Downloaded: " + str(user))


        followers_of_followers_ids[f['id']] = fof

serialize_json("data_ids/finale", "followers_of_followers.json", followers_of_followers_ids)

#Download dei following dei 10 following degli utenti random 
for utente in random_following_utenti_ids:
    for f in random_following_utenti_ids[utente]:
        fof = []
        print("scarico per " + str(utente))
        for item in tweepy.Cursor(
                api.friends,
                id=f['id'],
                skip_status=True,
                include_user_entities=False
        ).items(10):
            time.sleep(10)
            json_data = item._json
            user = {"id": json_data["id"]}
            fof.append(user)
            print("Downloaded: " + str(user))

        following_of_following_ids[f['id']] = fof

serialize_json("data_ids/finale", "following_of_following.json", following_of_following_ids)

ValueError: Sample larger than population or is negative

In [ ]:
#3

random_following_utenti_ids,following_of_following_ids={},{}

for utente in following_ids:
    random_following_utenti_ids[utente]=random.sample(following_ids[utente],5)

try:
    for utente in random_following_utenti_ids:
        for f in random_following_utenti_ids[utente]:
            fof=[]
            time.sleep(10)
            for item in tweepy.Cursor(
                api.following,
                id=f,
                skip_status=True,
                include_user_entities=False
            ).items(10):
                json_data=item._json
                user={"id":json_data["id"]}
                fof.append(user)

            following_of_following_ids[f["id"]]=fof
except tweepy.TweepError as error: #Le prime volte mi ha tornato TweepError:Unauthorized, ma ora non più
    print(error)
serialize_json("data_ids/finale","following_of_following.json",following_of_following_ids)

In [4]:
followers_ids=read_json("data_ids/followers_5_utenti.json")
following_ids=read_json("data_ids/following_5_utenti.json")
followers_of_followers_ids=read_json("data_ids/followers_of_followers.json")
following_of_following_ids=read_json("data_ids/following_of_following.json")
print("Json caricati")

Data read from path: data_ids/followers_5_utenti.json
Data read from path: data_ids/following_5_utenti.json
Data read from path: data_ids/followers_of_followers.json
Data read from path: data_ids/following_of_following.json
Json caricati


In [42]:
#unione in unico json finale senza ripetizioni, tanto abbiamo già estratto i follower random ed è più comodo per la chiamata api.show_friendship
#Logicamente, se non fosse stato per la consegna, avremmo avuto già le relazioni scaricando i followers dei 5 nodi principali

lista_json=[followers_ids,following_ids,followers_of_followers_ids,following_of_following_ids]
id_nodi_revisionati=[]
#print(lista_json)
for json_2 in lista_json:
    for utente in json_2:
      #  print(utente)
        for user_id in json_2[utente]:
            if not user_id in id_nodi_revisionati:
                id_nodi_revisionati.append(user_id['id'])

print(id_nodi_revisionati[:1])


[740410382]


In [47]:
#4 Scaricare i dati con api.get_user
id_nodi_revisionati = id_nodi_revisionati[:1]
nodes = {}
    
for user_id in id_nodi_revisionati: 
    node_info_of_id = []
    
    utente=api.get_user(id=user_id)._json
    
    node_infos= {} 
    node_infos["name"]=utente["name"]
    node_infos["screen_name"]=utente["screen_name"]
    node_infos["location"]=utente["location"]
    node_infos["followers_count"]=utente["followers_count"]
    node_infos["friends_count"]=utente["friends_count"]
    node_infos["statuses_count"]=utente["statuses_count"]
    node_infos["created_at"]=utente["created_at"]
    node_infos["profile_banner_url"]=utente["profile_banner_url"]
    node_info_of_id.append(node_infos)
    
    nodes[user_id] = node_info_of_id
    


{740410382: [{'name': 'ἀν-ἀρχή', 'screen_name': 'ColoursRiot', 'location': '', 'followers_count': 26, 'friends_count': 145, 'statuses_count': 874, 'created_at': 'Mon Aug 06 11:26:07 +0000 2012', 'profile_banner_url': 'https://pbs.twimg.com/profile_banners/740410382/1398799456'}]}


AttributeError: 'dict' object has no attribute 'dump'

In [48]:
print(nodes)    
serialize_json("data_ids", "nodes_of_twitter_graph.json", nodes)

{740410382: [{'name': 'ἀν-ἀρχή', 'screen_name': 'ColoursRiot', 'location': '', 'followers_count': 26, 'friends_count': 145, 'statuses_count': 874, 'created_at': 'Mon Aug 06 11:26:07 +0000 2012', 'profile_banner_url': 'https://pbs.twimg.com/profile_banners/740410382/1398799456'}]}


AttributeError: 'dict' object has no attribute 'dump'

In [ ]:
# Creazione dataframe dei Nodi  (da adattare ai json)
def create_node(user_follower):
    #lettura json dell'utente desiderato
    with open(user_follower+"_followers.json", "r", encoding="UTF-8") as read_file:
        data = json.load(read_file)
    #aggiunta dei nodi
    followers_nodes=[]
    for result in data[user_follower]:
        followers_nodes.append(result)  
    #df_nodi_grafo
    df_nodes = pd.DataFrame(followers_nodes)

In [ ]:
#Creazione dataframe degli archi di follow
def create_edges(follower): 
    #aggiunta degli archi
    friendships= []
    for result in friendship[relationship]
        if(result["source"]["followed_by"]== "true" | result["source"]["following"]== "true" ):
            if(result["source"]["followed_by"] == true):
                result["source"] = result["source"]["id"]
                result["followed"] = True
            if(result["source"]["following"] == true):
                result["target"] = result["target"]["id"]
                result["following"] = True
            friendships.append(result)

#inserimento nel dataframe
    df_edges = pd.DataFrame(friendships)
    return df_edges

In [ ]:
#Funzioni per la gestione dei dataframes (modifiche in base al formato dei json)

#Unione tra due dataframe
def dataframe_user(df_follower,df_following):
    frames= [df_follower,df_following]
    #unione dei dataframes tra followers e followings
    df_user = pd.concat(frames)
    return df_user

#creazione del dataframe dato un array di screen_names
def users_dfs(usernames):
    columns=['id','name','screen_name','location']  
    users_dfs = pd.DataFrame(columns=columns)
    for user in usernames:
        df_follower=user_followers_df(user)
        df_following=user_following_df(user)
        df_user=dataframe_user(df_follower,df_following)
        users_dfs=dataframe_user(users_dfs,df_user)
        
    return users_dfs

In [1]:
#5. Creazione del grafo
#-->In teoria i nodi arrivano dai json con il solito schema
#Aggiunta dei nodi al grafo
twitter_graph = nx.DiGraph(team="Loris Parata 144338, Francesco Arzon 142439, Lorenzo Dal Fabbro, Matteo Galvan")#non dobbiamo aggiungerci con tutte i parametri degli altri nodi?

for index,row in df_nodes.iterrows():
        twitter_graph.add_node(row["id"],
                        id= row["id"],
                        title= row["name"],
                        color ="#ffff00",
                        physics=False,#rende la visualizzazone del grafo più leggera
                        name=row['name'],
                        screen_name=row['screen_name'],
                        location=row['location'],
                        followers_count=row["followers_count"],
                        following_count=row["following_count"],
                        numbero_of_twitts=row["statuses_count"]#strano nome wtf
                        )            

#Aggiunta collegamenti al grafo
for user_id in users_id:#5 profili
    for utente in finale:#tutti gli altri
            status=api.show_friendship(id_source=user_id,id_target=utente["id"])
            if status[1].following:#da [tutti gli altri] a [5 profili]
                twitter_graph.add_edge(utente["id"],user_id,"follow_rec"=True if status[0].following else False) #nel caso volessimo evidenziare il follow reciproco

In [ ]:
#6 Creazione grafo interattivo con pyvis

def disegna_grafo(grafo):
    nt = Network(
        height ="50%",
        width = "50%",
        bgcolor="#222222",
        font_color="white",
        heading= grafo,
        directed=True,
    )

    nt.from_nx(grafo)
    nt.barnes_hut()
  # nt.set_edge_smooth("continuous") #cambia formato di visualizzazione degli archi
    neighbor_map = nt.get_adj_list()
    
    for node in nt.nodes:
            node["value"] = len(neighbor_map[node["id"]])
           # print(str(node["id"])+":"+ str(node["value"]))
         
    nt.show("grafo.html")


In [ ]:
#Main

#Array usernames dei nodi principali
usernames=["miccighel_","mizzaro","damiano10","eglu81","KevinRoitero"]
df_users= users_dfs(usernames)
#Creazione del grafo
twitter_graph=following_graph(df_users)
#print(fg.nodes().data())
twitter_graph.number_of_edges()
#disegna_grafo(following_graph(df.loc[df['id'] == 1527461]))
#disegna_grafo(fg)

In [ ]:
#7 Proprietà del grafo
undirected_twitter_graph= nx.to_undirected(twitter_graph)

if nx.is_connected(undirected_twitter_graph):
    print("Il grafo è connesso.")
else:
    print("Il grafo non è connesso.")
    
if nx.is_bipartite(undirected_twitter_graph):
    print("Il grafo è bipartito.")
else:
    print("Il grafo non è bipartito.")

In [ ]:
#8 Distanze sul grafo
centro = nx.center(twitter_graph)
print(centro)
diametro = nx.diameter(twitter_graph)
print(diametro)
raggio = nx.radius(twitter_graph)
print(raggio)

In [2]:
#9 Misure di centralità sul grafo
btw_centrality = nx.betweenness_centrality(twitter_graph)
print(btw_centrality)
cls_centrality = nx.closeness_centrality(twitter_graph)
print(cls_centralitu)
dg_centrality = nx.degree_centrality(twitter_graph)
print(dg_centrality)

in_centrality= nx.in_degree_centrality(twitter_graph)
print(in_centrality)
out_centrality= nx.out_degree_centrality(twitter_graph)
print(out_centrality)
#valori di centralità
undirect_centralities=[btw_centrality,cls_centrality,dg_centrality]
direct_centralities=[in_centrality,out_centrality]

pagerank= nx.pagerank(twitter_graph)
pagerank=sorted(pagerank.items(),key=lambda x:x[1],reverse=True)
centralities= [btw_centrality,cls_centrality,dg_centrality,in_centrality,out_centrality]
print(pagerank[:5])

hits= nx.hits(twitter_graph)
hits_hubs=hits[0]
hits_hub= sorted(hits_hubs.items(),key=lambda x:x[1],reverse=True)
print(hits_hub[:5])
hits_authorities=hits[1]
hits_authorities=sorted(hits_authorities.items(),key=lambda x:x[1],reverse=True)
print(hits_authorities[:5])

NameError: name 'twitter_graph' is not defined

In [ ]:
#Sottografo del nodo damiano10
sub_graph_damiano10 = nx.ego_graph(twitter_graph, "damiano10")
disegna_grafo(sub_graph_damiano10)
print(help(nx.clique))
clique.max_clique(sub_graph_damiano10)
print(nx.large_clique_size(sub_graph_damiano10))

In [ ]:
#11 copertua minima degli archi
nx.min_edge_cover(twitter_graph)

#12 Small-world-ness
nx.omega(twitter_graph)
nx.sigma(twitter_graph)

In [ ]:
#13 Correlazione di Pearson e di Kendall

columns=["name","value"]
centralita_df = pd.DataFrame(columns=columns)
rows = [["btw_centrality",btw_centrality],["cls_centrality",cls_centrality],["dg_centrality",dg_centrality],["in_centrality",in_centrality],["out_centrality",out_centrality]]
#Creazione df contenente tipologia_misura : valore
for row in rows:
    centralita_df.loc[len(centralita_df)] = row
#print(centralita_df)

#Creazione df contenente  tipologia_misura_1 , tipologia_misura_2, valore_correlazione_1 , valore_correlazione_2
columns=["name_1","name_2","pearson", "kendall"]
result= pd.DataFrame(columns=columns)

for index, param_1 in centralita_df.iterrows():
    for index, param_2 in centralita_df.iterrows():
        if(param_1["name"] != param_2["name"]):
            name_1= param_1["name"]
            name_2= param_2["name"]
            array_1= np.array(list(param_1["value"].values()), dtype=float)
            array_2= np.array(list(param_2["value"].values()), dtype=float)
            pearson= stats.pearsonr(array_1,array_2)
            kendall= stats.kendalltau(array_1,array_2)
            new_row = {
            'name_1': name_1,
            'name_2': name_2,
            'pearson': pearson,
            'kendall': kendall
                }
            result = result.append(new_row, ignore_index=True)

print(result)